# Spotify Mood Session

## 1. Algorithm

a. **First Selection**. User selects a song $s_0$ from the set of all songs $S$ using some kind of search method (by name / artist) and plays that song.

b. **Played List**.  Let $P$ be a new list of played songs and let $P \gets s_0$   

c. **Cold Start**. Create a bootstrap random selection of $n$ songs from $S$ called $R$. Calculate the distance between each song and the first selection and select the closest to the chosen song . Formally:

$$
\forall s \in R \text{ dist}(s_i, s_0)
$$

*potential parameter: use a value to add the closest n songs to the initial bootstrap played list*

d. **Rating**. For every song that a user plays for more than 30 seconds, the song is rated $\text{pos}$, songs that are skipped within this time period are rated $\text{neg}$.  Songs that have not been played are rated $\text{non}$

e. **Recommending**.  Take the current set of vectors and select the next song based on some sort of agregated mean or nearest neighbour system.

repeat (d) and (e) ad infinitum.

Formally:

$$
\text{let: } s_0 \gets \text{ user}(S) \\
\text{let: } P = \{ s_0 \} \\
\text{let: } R \subset S \\
\text{let: } D = \emptyset \\ 
\text{for } s_i \text{ in } R:\\
D_i = \text{dist}(s_i, s_0) \\
\text{end for }\\
\text{select}(\text{max}(D))\\
$$

In [2]:
import json
import pandas as pd
import os
from IPython.display import display

# create list of playlists
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
print(playlists.shape)

(5000, 12)


In [3]:
# Add all songs from the playlists into a single list of songs
songs = [] 
for i in range(len(playlists)):
    tracks = playlists.iloc[1].loc['tracks']
    for track in tracks:
        songs.append(track)

print(len(songs))
print(songs[0])

130000
{'pos': 0, 'artist_name': 'Erykah Badu', 'track_uri': 'spotify:track:6AaQoliicVSZhobw0OSSM0', 'artist_uri': 'spotify:artist:7IfculRW2WXyzNQ8djX8WX', 'track_name': 'Hello', 'album_uri': 'spotify:album:5q1BjSadVkASNdJ4neVmt6', 'duration_ms': 319958, 'album_name': 'But You Caint Use My Phone'}


In [13]:
# Remove duplicate songs by URI
occurances = {}
for i in range(len(songs)):
    
    uri = songs[i]['track_uri']
    if uri in occurances:
        occurances[uri] += 1
    else:
        occurances[uri] = 1

print(len(occurances.keys()))

26
